# Normal mode and harmonic fitting with reductions 

by Willi Rath (wrath@geomar.de) and Kristin Burmeister (kristin.burmeister@sams.ac.uk)

We'll use orthogonality between functions to fit (vertical) normal modes and (temporal) harmonics with reduction operations (summing along axis).

Let $b_n(z)$ be vertical normal modes with 
$$\int {\rm d}z\, b_n(z) \cdot b_m(z) = \delta_{n,m}$$ 
and $h_{T,\tau}(t)$ be temporal harmonic modes with period $T$ and phase $\tau$ which fulfil 
$$\int {\rm d}t\, h_{T,0}(t) \cdot h_{T/2,0}(t) = 0$$ 
$$\int {\rm d}t\, h_{T,0}(t) \cdot h_{T,0}(t) = 1$$ 

Then, we could compose a signal $s(t, z)$ with different normal modes each having a separate annual and semi-annual cycle as: 
$$s(t, z)= \sum_n \alpha^a_n \cdot b_n(z) \cdot h_{365d, \tau^a_n}(t) + \sum_n \alpha^s_n \cdot b_n(z) \cdot h_{365d/2.0, \tau^s_n}(t)$$ 
where $\alpha^a_n$ are the amplitudes of the annual cycle of the vertical mode $n$, $\alpha^s_n$ are the amplitudes of the semi-annual cycle of the vertical mode $n$, $\tau^a_n$ is the phase shift of the annual cycle of vertical mode $n$, and $\tau^s_n$ is the phase shift of the semi-annual cycle of vertical mode $n$. 
   
The time-variability of the vertical mode $n$ can be diagnosed using a time integral: 
$$\alpha^a_1 \cdot h_{365d,\tau^a_1}(t) + \alpha^a_1 \cdot h_{365d/2.0,\tau^a_1}(t) = \int {\rm d}z\, b_1(z) \cdot s(t,z) \equiv s_1(t)$$ 
$$\alpha^a_2 \cdot h_{365d,\tau^a_2}(t) + \alpha^a_2 \cdot h_{365d/2.0,\tau^a_2}(t) = \int {\rm d}z\, b_2(z) \cdot s(t,z) \equiv s_2(t)$$ 
$$...$$ 
   
The phase and amplitude of $s_n(t)$ can be diagnosed by projecting on a normalized annual ${\rm e}^{2i\pi/365d \cdot t}$ or semi-annual ${\rm e}^{4i\pi/365d \cdot t}$: 
   
$$\alpha^a_1 \propto \left|\int {\rm d}t\, {\rm e}^{2i\pi/365d \cdot t} s_1(t)\right|$$ 
   
$$\alpha^a_2 \propto \left|\int {\rm d}t\, {\rm e}^{2i\pi/365d \cdot t} s_2(t)\right|$$ 
   
$$...$$ 
   
$$\alpha^s_1  
  \propto \left|\int {\rm d}t\, {\rm e}^{4i\pi/365d \cdot t} s_1(t)\right|$$ 
   
$$\alpha^s_2  
  \propto \left|\int {\rm d}t\, {\rm e}^{4i\pi/365d \cdot t} s_2(t)\right|$$ 
   
$$...$$ 
   
and 
   
$$\tau^a_1 = {\rm arg}\left(\int {\rm d}t\, {\rm e}^{2i\pi/365d \cdot t} s_1(t)\right)$$ 
$$\tau^a_2 = {\rm arg}\left(\int {\rm d}t\, {\rm e}^{2i\pi/365d \cdot t} s_2(t)\right)$$ 
$$...$$ 
   
$$\tau^s_1 = {\rm arg}\left(\int {\rm d}t\, {\rm e}^{4i\pi/365d \cdot t} s_1(t)\right)$$ 
$$\tau^s_2 = {\rm arg}\left(\int {\rm d}t\, {\rm e}^{4i\pi/365d \cdot t} s_2(t)\right)$$ 
$$...$$

## Tech preample

In [1]:
%matplotlib inline
import os
import pandas as pd
import xarray as xr
import numpy as np
import pandas as pd
from pathlib import Path

## Input parameters 

In [2]:
in_modes = Path('../data/INALT20_dynmodes')
file_modes_obs = 'Updated_ctd_ta_23W_modes.nc'

## Define functions

In [3]:
# fitting with reductions
def fit_vert_modes(pstruc,data,dz,dim='depth'):
    return (pstruc*data*dz).sum(dim)

def harmonic_cycle(t, T=1, phi=0):
    """Create harmonic cycles."""
    return np.sin(2 * np.pi / T * (t + phi))

def normalize(x=None, y=None):
    return y / np.linalg.norm(y) / (x.max() - x.min()) ** 0.5

def harmonic_proj(t=None, T=None, dt=None, signal=None):
    #     harmonic_mode = (
    #         normalize(t, harmonic_cycle(t, T=T, phi=0))
    #         + 1j * normalize(t, harmonic_cycle(t, T=T, phi=T / 4.0))
    #     ) / (2 ** 0.5)
    harmonic_mode = normalize(
        t,
        harmonic_cycle(t, T=T, phi=0) + 1j * harmonic_cycle(t, T=T, phi=T / 4.0)
    )
    return (signal * xr.DataArray(harmonic_mode, dims='time')* dt**0.5).sum('time') #needed to add dt**0.5 to get amplitudes right if dt!=1

def harmonic_phase(t=None, T=None, dt=None, signal=None):
    proj = harmonic_proj(t=t, T=T, dt=dt, signal=signal)
    phi = np.arctan2(np.imag(proj), np.real(proj)) * T / np.pi / 2
    phi.attrs['name'] = 'Phase'
    phi.attrs['units'] = 'days'
    return phi

def harmonic_amplitude(t=None, T=None, dt=None, signal=None):
    proj = harmonic_proj(t=t, T=T, dt=dt, signal=signal)
    return 2 * np.abs(proj)

def harm_fit(s_n):
    time_ordinal = np.array([pd.to_datetime(x).toordinal() for x in s_n.time.values])
    time_ordinal -= time_ordinal[0]
    s_n.coords['time_ordinal']=(['time'],time_ordinal)
    dt = time_ordinal[1]-time_ordinal[0]

    ah_pha = harmonic_phase(s_n.time_ordinal, 365,dt, s_n)
    ah_amp = harmonic_amplitude(s_n.time_ordinal, 365,dt, s_n)
    sh_pha = harmonic_phase(time_ordinal, 365 / 2.0,dt, s_n)
    sh_amp = harmonic_amplitude(time_ordinal, 365 / 2.0,dt, s_n)
    return xr.merge((ah_pha.rename('ah_pha'), ah_amp.rename('ah_amp'), sh_pha.rename('sh_pha'), sh_amp.rename('sh_amp')))

## Load data

In [4]:
modes_obs = xr.open_dataset(in_modes/file_modes_obs)
modes_obs = modes_obs.sel(mode=slice(1,5))
modes_obs

<xarray.Dataset>
Dimensions:  (depth_p: 450, depth_w: 451, mode: 5)
Coordinates:
  * depth_w  (depth_w) float64 0.0 10.0 20.0 30.0 ... 4.48e+03 4.49e+03 4.5e+03
  * depth_p  (depth_p) float64 5.0 15.0 25.0 ... 4.475e+03 4.485e+03 4.495e+03
  * mode     (mode) int64 1 2 3 4 5
Data variables:
    wstruc   (mode, depth_w) float64 ...
    pstruc   (mode, depth_p) float64 ...
    rstruc   (mode, depth_w) float64 ...
    Nsq      (depth_w) float64 ...
    c        (mode) float64 ...
Attributes:
    description:  Vertical structure functions for stratification from mean C...
    history:      Created by https://github.com/Kristin-2002/Wind_forcing/blo...
    author:       Kristin Burmeister, kristin.burmeister@sams.ac.uk, ORCID 00...

### Test if vertical modes are correctly normalized

In [5]:
print(f"THEORY: Cumsum of $M_N$x$M_N$=1, $M_N$x$M_M$=0, $M_M$x$M_M$=1")
for modNo in range(1,5):
    normal_mode_1 = modes_obs.pstruc.sel(mode=modNo)
    normal_mode_2 = modes_obs.pstruc.sel(mode=modNo+1)
    z = -modes_obs.depth_p
    dz = modes_obs.depth_p.diff('depth_p').isel(depth_p=0)
    m11 = '{0:.3g}'.format((normal_mode_1 * normal_mode_1*dz).cumsum().squeeze()[-1].values)
    m12 = '{0:.3g}'.format((normal_mode_1 * normal_mode_2*dz).cumsum().squeeze()[-1].values)
    m22 = '{0:.3g}'.format((normal_mode_2 * normal_mode_2*dz).cumsum().squeeze()[-1].values)
    print(f"Mode {(modNo)} and {modNo+1}: M{modNo}xM{modNo}={m11}, M{modNo}xM{modNo+1}={m12}, M{modNo+1}xM{modNo+1}={m22}")

THEORY: Cumsum of $M_N$x$M_N$=1, $M_N$x$M_M$=0, $M_M$x$M_M$=1
Mode 1 and 2: M1xM1=1, M1xM2=-1.48e-12, M2xM2=1
Mode 2 and 3: M2xM2=1, M2xM3=-5.2e-14, M3xM3=1
Mode 3 and 4: M3xM3=1, M3xM4=-2.82e-12, M4xM4=1
Mode 4 and 5: M4xM4=1, M4xM5=1.24e-12, M5xM5=1


## Create artificial dataset

In [6]:
%%time
amp_in_ah = (modes_obs.mode*0+[1.2, -4.3, 5, 7.3, -.3]).rename('coeffs')
amp_in_sh = (modes_obs.mode*0+[2.3, 6 , -1.2, 1, -.8]).rename('coeffs')
dt = 5
t=np.arange(0,10 * 365,dt)
phase_in_ah = 0
phase_in_sh = 80

xN,yN = 10,5
lon = xr.DataArray(np.arange(xN),dims='lon',coords={'lon':np.arange(xN)})
lat = xr.DataArray(np.arange(yN),dims='lat',coords={'lat':np.arange(yN)})

# create artificial dataset
vertical_data1 = (amp_in_ah * modes_obs.pstruc).sum('mode')
vertical_data2 = (amp_in_sh * modes_obs.pstruc).sum('mode')

annual_signal = xr.DataArray(harmonic_cycle(t, T=365, phi=phase_in_ah),dims='time',coords={'time':pd.date_range('2000-01-01','2009-12-28',freq='5D')})
semi_annual_signal = xr.DataArray(harmonic_cycle(t, T=182.5, phi=phase_in_sh),dims='time',coords={'time':pd.date_range('2000-01-01','2009-12-28',freq='5D')})
data = (vertical_data1*annual_signal + vertical_data2*semi_annual_signal)*(lon*0+1)*(lat*0+1)
display(data)

<xarray.DataArray (depth_p: 450, time: 730, lon: 10, lat: 5)>
array([[[[ 0.22842077,  0.22842077,  0.22842077,  0.22842077,
           0.22842077],
         [ 0.22842077,  0.22842077,  0.22842077,  0.22842077,
           0.22842077],
         [ 0.22842077,  0.22842077,  0.22842077,  0.22842077,
           0.22842077],
         ...,
         [ 0.22842077,  0.22842077,  0.22842077,  0.22842077,
           0.22842077],
         [ 0.22842077,  0.22842077,  0.22842077,  0.22842077,
           0.22842077],
         [ 0.22842077,  0.22842077,  0.22842077,  0.22842077,
           0.22842077]],

        [[ 0.15278482,  0.15278482,  0.15278482,  0.15278482,
           0.15278482],
         [ 0.15278482,  0.15278482,  0.15278482,  0.15278482,
           0.15278482],
         [ 0.15278482,  0.15278482,  0.15278482,  0.15278482,
           0.15278482],
...
         [ 0.04009326,  0.04009326,  0.04009326,  0.04009326,
           0.04009326],
         [ 0.04009326,  0.04009326,  0.04009326,  0.04009326,
           0.04009326],
         [ 0.04009326,  0.04009326,  0.04009326,  0.04009326,
           0.04009326]],

        [[ 0.0308065 ,  0.0308065 ,  0.0308065 ,  0.0308065 ,
           0.0308065 ],
         [ 0.0308065 ,  0.0308065 ,  0.0308065 ,  0.0308065 ,
           0.0308065 ],
         [ 0.0308065 ,  0.0308065 ,  0.0308065 ,  0.0308065 ,
           0.0308065 ],
         ...,
         [ 0.0308065 ,  0.0308065 ,  0.0308065 ,  0.0308065 ,
           0.0308065 ],
         [ 0.0308065 ,  0.0308065 ,  0.0308065 ,  0.0308065 ,
           0.0308065 ],
         [ 0.0308065 ,  0.0308065 ,  0.0308065 ,  0.0308065 ,
           0.0308065 ]]]])
Coordinates:
  * depth_p  (depth_p) float64 5.0 15.0 25.0 ... 4.475e+03 4.485e+03 4.495e+03
  * time     (time) datetime64[ns] 2000-01-01 2000-01-06 ... 2009-12-24
  * lon      (lon) int64 0 1 2 3 4 5 6 7 8 9
  * lat      (lat) int64 0 1 2 3 4

CPU times: user 89.8 ms, sys: 21.1 ms, total: 111 ms
Wall time: 110 ms


## Fit vertical and harmonic modes

In [7]:
sn_test = fit_vert_modes(modes_obs.pstruc,data,10,dim='depth_p').load()
harm_test = harm_fit(sn_test)

print('amp_in_ah  =',amp_in_ah.values)
print('amp_fit_ah =',harm_test.sel(lat=0,lon=0).ah_amp.values)

print('phase_in_ah  =',phase_in_ah)
print('phase_fit_ah =',harm_test.sel(lat=0,lon=0).ah_pha.values)

print('amp_in_sah  =',amp_in_sh.values)
print('amp_fit_sah =',harm_test.sel(lat=0,lon=0).sh_amp.values)

print('phase_in_sah  =',phase_in_sh)
print('phase_fit_sah =',harm_test.sel(lat=0,lon=0).sh_pha.values)

amp_in_ah  = [ 1.2 -4.3  5.   7.3 -0.3]
amp_fit_ah = [1.20082276 4.30294823 5.00342818 7.30500514 0.30020569]
phase_in_ah  = 0
phase_fit_ah = [-2.65606106e-14 -1.82500000e+02  1.65154128e-14  1.15322160e-14
 -1.82500000e+02]
amp_in_sah  = [ 2.3  6.  -1.2  1.  -0.8]
amp_fit_sah = [2.30157696 6.00411382 1.20082276 1.00068564 0.80054851]
phase_in_sah  = 80
phase_fit_sah = [ 80.    80.   -11.25  80.   -11.25]


## The questions of the unit

The fitted parameters agree well with the input parameters used to create the artifical velocity time series. To achieved that I needed to add a `dt**0.5` in the `harmonic_proj` function.

Doesn't that mean that the amplitudes (in and fitted) have the same unit (i.e m/s). The vertical modes are dimensionless as well as the harmonic time series. The amplitude (`amp_in_ah, amp_in_sh`) is what makes my velocity field a velocity field (should have the unit m/s). As my fit returns the same values they should also have the same unit?